# Data Diagnosis

In [11]:
import pandas as pd

In [12]:
df_swaps = pd.read_csv('../data/raw/case_data_science_charging_ops___battery_swap_2025-09-26T17_45_41.358859844Z.csv')
df_stations = pd.read_csv('../data/raw/case_data_science_charging_ops___swap_stations_info_2025-09-26T13_21_20.332938629Z.csv')
df_traffic = pd.read_csv('../data/raw/ds_case_data_2025-09-26T12_43_22.934122897Z.csv')

C:\Users\vican\AppData\Local\Temp\ipykernel_15168\4079410079.py:3: DtypeWarning: Columns (3) have mixed types. Specify dtype option on import or set low_memory=False.
  df_traffic = pd.read_csv('../data/raw/ds_case_data_2025-09-26T12_43_22.934122897Z.csv')


In [13]:
def diagnosis(df):
    print('## Dataset Size ##')
    print(f'{len(df)}')
    print('\n ## Column Types ##')
    print(df.dtypes)
    print('\n ## NA Counts ##')
    print(df.isna().sum())
    print('\n ## Duplicates ##')
    print(df.duplicated().sum())

## Swaps

In [15]:
df_swaps.head()

,created_at,ended_at,cabinet_id,swap_station_id,battery_out_id,battery_in_id,rider_id,status,battery_in_level,battery_out_level
0,"January 1, 2025, 12:00 AM","January 1, 2025, 12:00 AM",785,834,"15,506","13,900","28,714",completed,8,100
1,"January 1, 2025, 12:00 AM","January 1, 2025, 12:01 AM",476,834,"14,665","8,087","30,510",completed,20,100
2,"January 1, 2025, 12:00 AM","January 1, 2025, 12:05 AM",896,"1,155","14,586","14,663","22,368",completed,68,100
3,"January 1, 2025, 12:01 AM","January 1, 2025, 12:01 AM",59,553,"14,806","15,492","25,990",completed,20,100
4,"January 1, 2025, 12:01 AM","January 1, 2025, 12:02 AM",785,834,"16,789","15,015","28,714",completed,10,100


In [16]:
diagnosis(df_swaps)

## Dataset Size ##
383739

 ## Column Types ##
created_at           object
ended_at             object
cabinet_id           object
swap_station_id      object
battery_out_id       object
battery_in_id        object
rider_id             object
status               object
battery_in_level      int64
battery_out_level     int64
dtype: object

 ## NA Counts ##
created_at               0
ended_at              8811
cabinet_id               7
swap_station_id          0
battery_out_id       12762
battery_in_id        12762
rider_id                 0
status                   0
battery_in_level         0
battery_out_level        0
dtype: int64

 ## Duplicates ##
1300


In [17]:
df_swaps.status.value_counts()

completed                        370303
TIMEOUT                            5928
ERROR                              4046
pending                            2015
CONNECTION_TIMEOUT                  788
SWAP_SUCCESS_DOOR_LEFT_OPENED       637
CLOSE_SECOND_SLOT                    15
canceled                              5
expired                               2
Name: status, dtype: int64

In [18]:
df_swaps[['battery_in_level', 'battery_out_level']].astype(int).describe().round()

,battery_in_level,battery_out_level
count,383739.0,383739.0
mean,33.0,92.0
std,20.0,19.0
min,0.0,0.0
25%,20.0,96.0
50%,27.0,100.0
75%,45.0,100.0
max,100.0,100.0


### Plan
- Remove duplicates;
- Remove null values for relevant columns (cabinet_id);
- Disregard columns "battery_in_id" and "battery_out_id" -> too many null data;
- Process date columns;
- Replace "," in numeric columns;
- Normalize data types;
- Normalize status column;
- Create "battery charged" feature;
- Create time features (duration, year, month, etc).

## Stations

In [21]:
df_stations.head()

,swap_station_id,Endereço,latitude,longitude
0,241,"Av. das Nações, 2851, Ourilândia do Norte - PA...",-6.75,-51.07
1,241,"Av. das Nações, 2851, Ourilândia do Norte - PA...",-6.75,-51.07
2,1,"R. Santa Justina, 434 - Vila Olímpia, São Paul...",-23.60,-46.68
3,274,"R. Azevedo Soares, 701 - Vila Gomes Cardim, Sã...",-23.55,-46.57
4,901,"R. Comandante Taylor, 490 - Ipiranga, São Paul...",-23.60,-46.60


In [22]:
diagnosis(df_stations)

## Dataset Size ##
482

 ## Column Types ##
swap_station_id     object
Endereço            object
latitude           float64
longitude          float64
dtype: object

 ## NA Counts ##
swap_station_id    0
Endereço           0
latitude           0
longitude          0
dtype: int64

 ## Duplicates ##
329


In [23]:
df_stations = df_stations.drop_duplicates()
len(df_stations[df_stations['Endereço'].str.contains('- SP')])

151

### Plan
- Remove duplicates;
- Normalize columns names (Endereço);
- Name stations;
- Create geospatial features.

In [56]:
df_stations = df_stations[~df_stations['Endereço'].str.contains('- PA')][:].reset_index(drop=True)
df_stations = df_stations.drop_duplicates()
df_stations['neighborhood'] = df_stations['Endereço'].str.extract(r'-\s*([^,]+),')

In [62]:
df_stations[df_stations['neighborhood']=='Centro']

,swap_station_id,Endereço,latitude,longitude,neighborhood
8,755,"R. Baraldi, 1149 - Centro, São Caetano do Sul ...",-23.61,-46.57,Centro
44,"1,153","R. Sen. Flaquer, 825 - Centro, Santo André - S...",-23.66,-46.52,Centro
67,"1,535","Av. Antônio Piranga, 615 - Centro, Diadema - S...",-23.69,-46.62,Centro
77,"1,041","R. Secondo Modolin, 395 - Centro, São Bernardo...",-23.71,-46.56,Centro
94,"1,231","Av. Rotary, 435 - Centro, São Bernardo do Camp...",-23.72,-46.55,Centro
101,"1,534","Av. Pres. Kennedy, 334 - Centro, Diadema - SP,...",-23.68,-46.63,Centro
118,"1,412","Av. Pref. Valdírio Prisco, 1360 - Centro, Ribe...",-23.71,-46.41,Centro
145,"1,577","R. Luís Mariani, 148 - Centro, Mauá - SP, 0939...",-23.67,-46.46,Centro


In [86]:
df_stations[df_stations.neighborhood.isna()==True]

,swap_station_id,Endereço,latitude,longitude,neighborhood
123,"1,654","Santiago, Santiago Metropolitan Region, Chile",-33.45,-70.67,NaN


In [58]:
df_stations['neighborhood'].value_counts()

Centro              8
Penha de França     4
Pinheiros           4
Rudge Ramos         3
Santana             3
                   ..
Vila Santa Luzia    1
Morumbi             1
Serraria            1
Vila Prel           1
Demarchi            1
Name: neighborhood, Length: 120, dtype: int64

## Traffic

In [26]:
df_traffic.head()

,lat,lng,week_observed,observations
0,-23.63,-46.68,"February 10, 2025","4,791"
1,-23.63,-46.68,"February 10, 2025","4,698"
2,-23.52,-46.44,"February 10, 2025","4,661"
3,-23.67,-46.61,"February 10, 2025","4,622"
4,-23.61,-46.67,"February 10, 2025","4,620"


In [27]:
diagnosis(df_traffic)

## Dataset Size ##
1048575

 ## Column Types ##
lat              float64
lng              float64
week_observed     object
observations      object
dtype: object

 ## NA Counts ##
lat              0
lng              0
week_observed    0
observations     0
dtype: int64

 ## Duplicates ##
974089


In [28]:
(
    df_traffic
    .dropna(subset='observations')
    .drop_duplicates()
    .observations.astype(str).str.replace(',', '')
    .astype(int)
    .describe()
    .round()
)

count    74486.0
mean        97.0
std        252.0
min          9.0
25%         19.0
50%         37.0
75%         79.0
max       4791.0
Name: observations, dtype: float64

In [102]:
df_traffic['week_observed']

0          February 10, 2025
1          February 10, 2025
2          February 10, 2025
3          February 10, 2025
4          February 10, 2025
                 ...        
1048570    February 10, 2025
1048571    February 10, 2025
1048572    February 10, 2025
1048573    February 10, 2025
1048574    February 10, 2025
Name: week_observed, Length: 1048575, dtype: object

### Plan
- Process date column;
- Normalize observations column;
- Create labels for traffic level;
- Find nearby stations;
- Duplicates: sum or remove?